In [32]:
# establish connection
from os import getenv
from dotenv import load_dotenv
import pyodbc
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=localhost\\DOKUSSERVER;"
                      "Database=Lernperiode12;"
                      "Trusted_Connection=yes;")

cursor = cnxn.cursor()

In [33]:
cursor.execute("SELECT MAX(timestamp) FROM WeatherMeasurement")
last_saved_time = cursor.fetchone()[0]

In [34]:
from datetime import datetime, timedelta
if last_saved_time is None:
    last_saved_time = datetime.now() - timedelta(hours=48)
start = last_saved_time - timedelta(hours=48)
end = datetime.now()

start_str = start.date().isoformat()
end_str   = end.date().isoformat()

In [36]:
print(last_saved_time)

2025-12-09 23:00:00


In [37]:
print(start_str)

2025-12-07


In [38]:
print(end_str)

2025-12-10


In [42]:
# request
import requests
import json
from datetime import datetime

def get_weather_data(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "precipitation"
        ],
        "past_hours": 48,
        "forecast_days": 0,
        "timezone": "auto"
    }




    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print("Fetch failed:", e)
        return None


# Coordinates for Baden, Switzerland
baden_lat = 47.475361
baden_lon = 8.306372

weather = get_weather_data(baden_lat, baden_lon)
print(weather)

{'latitude': 47.48, 'longitude': 8.299999, 'generationtime_ms': 0.36585330963134766, 'utc_offset_seconds': 3600, 'timezone': 'Europe/Zurich', 'timezone_abbreviation': 'GMT+1', 'elevation': 394.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'wind_speed_10m': 'km/h', 'precipitation': 'mm'}, 'hourly': {'time': ['2025-12-08T19:00', '2025-12-08T20:00', '2025-12-08T21:00', '2025-12-08T22:00', '2025-12-08T23:00', '2025-12-09T00:00', '2025-12-09T01:00', '2025-12-09T02:00', '2025-12-09T03:00', '2025-12-09T04:00', '2025-12-09T05:00', '2025-12-09T06:00', '2025-12-09T07:00', '2025-12-09T08:00', '2025-12-09T09:00', '2025-12-09T10:00', '2025-12-09T11:00', '2025-12-09T12:00', '2025-12-09T13:00', '2025-12-09T14:00', '2025-12-09T15:00', '2025-12-09T16:00', '2025-12-09T17:00', '2025-12-09T18:00', '2025-12-09T19:00', '2025-12-09T20:00', '2025-12-09T21:00', '2025-12-09T22:00', '2025-12-09T23:00', '2025-12-10T00:00', '2025-12-10T01:00', '2025-12-10T02:00', '2025

In [40]:
# getting only new rows with pandas
import pandas as pd
import pytz
df["time"] = pd.to_datetime(df["time"], utc=True)
last_saved_time = pd.Timestamp(last_saved_time, tz="UTC")
df_new = df[df["time"] > last_saved_time]





In [41]:
df_new.tail(5)

,time,temperature_2m,relative_humidity_2m,wind_speed_10m,precipitation
427,2025-12-26 14:00:00+00:00,NaN,NaN,NaN,NaN
428,2025-12-26 15:00:00+00:00,NaN,NaN,NaN,NaN
429,2025-12-26 16:00:00+00:00,NaN,NaN,NaN,NaN
430,2025-12-26 17:00:00+00:00,NaN,NaN,NaN,NaN
431,2025-12-26 18:00:00+00:00,NaN,NaN,NaN,NaN


In [14]:
# weather data template
SQL_API_STATEMENT = """
INSERT INTO WeatherMeasurement (
    timestamp,
    temperature,
    humidity,
    windspeed,
    precipitation
) VALUES
( ?, ?, ?, ?, ?)
"""

In [15]:
# insert current weather data
from datetime import datetime

for _, row in df_new.iterrows():
    cursor.execute(
        SQL_API_STATEMENT,
        (
            row["time"],
            row["temperature_2m"],
            row["relative_humidity_2m"],
            row["wind_speed_10m"],
            row["precipitation"]
        )
    )
cnxn.commit()


In [16]:
print(df_new.tail())
print(len(df_new))


                        time  temperature_2m  relative_humidity_2m  \
43 2025-12-09 19:00:00+00:00             5.3                   100   
44 2025-12-09 20:00:00+00:00             5.2                   100   
45 2025-12-09 21:00:00+00:00             5.7                    98   
46 2025-12-09 22:00:00+00:00             6.4                    98   
47 2025-12-09 23:00:00+00:00             6.1                    98   

    wind_speed_10m  precipitation  
43             3.7            0.0  
44             3.2            0.0  
45             4.7            0.0  
46             4.2            0.0  
47             5.8            0.0  
48


In [ ]:
# check last data time
# get data from 48 hours
    # define end and start
# compare pulled data with last inserted time
# leave only the data that is higher than max time from db
# insert
# if there is a gap
# then inform there's missing data because pulled from only 48h
# missing: x hours